In [1]:
from pandas_datareader import data as pdr
import yfinance as yfin
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import import_ipynb

class make_feature:
    def __init__(self, symbol, start, end):
        self.symbol = symbol
        self.start = start
        self.end = end
        self.data_load = self.data_(self.symbol, self.start, self.end)

        self.linear = nn.Linear(11,10)
        self.tanh = nn.Tanh()

    def data_(self, symbol, start, end):
        yfin.pdr_override()
        data = pdr.get_data_yahoo(symbol, start=start, end=end)
        return data

    def make_lt_feature(self, data):
        lt_feature_list = []
        time_terms = [5,10,15,20,25,30]
        for time_term in time_terms:

            data_len = len(data.loc[:, "Adj Close"])
            hidden = [sum(data.loc[:, "Adj Close"][data_len - i - time_term:data_len - i]) for i in range(data_len)]

            lis_ = []
            for i in range(len(hidden)):
                length = len(hidden)
                a = hidden[length - i - 1]
                lis_.append(a)
            len(lis_)

            zdk = np.array(lis_) / time_term * np.array(data.loc[:, "Adj Close"]) - 1
            lt_feature_list.append(zdk)
        return np.array(lt_feature_list).T

    def tgt_make(self, data):
        tgt = np.where(data.loc[:, "Adj Close"] >= data.loc[:, "Adj Close"].shift(periods=1, axis=0), 1.0, 0.0)
        return tgt

    def all_feature(self):

        data = self.data_load
        lt_feature = self.make_lt_feature(data)

        z_open = (data.loc[:, "Open"] / data.loc[:, "Close"] - 1).to_numpy()
        z_high = (data.loc[:, "High"] / data.loc[:, "Close"] - 1).to_numpy()
        z_low = (data.loc[:, "Low"] / data.loc[:, "Close"] - 1).to_numpy()
        z_close = (data.loc[:, "Close"] / data.loc[:, "Close"].shift(periods=1, axis=0) - 1).to_numpy()
        z_adjclose = (data.loc[:, "Adj Close"] / data.loc[:, "Adj Close"].shift(periods=1, axis=0) - 1).to_numpy()



        tgt = self.tgt_make(data)

        df = np.column_stack((z_open, z_high, z_low, z_close, z_adjclose,lt_feature,tgt))

        return pd.DataFrame(df[29:])



if __name__=="__main__":
    mf = make_feature('^IXIC',"2007-01-01","2016-12-31")
    df = mf.all_feature()
    print(df)
    df.to_csv('C:\\Users\\USER\\JupyterProjects\\AdvLSTM\\data\\kdd17\\ourpped1\\IXIC.csv', encoding='utf-8',index=False, header=False)




[*********************100%***********************]  1 of 1 completed
            0         1         2         3         4             5   \
0    -0.008174  0.002464 -0.008174  0.011586  0.011586  6.144869e+06   
1    -0.002907  0.000509 -0.004641  0.003504  0.003504  6.170613e+06   
2    -0.001935  0.000645 -0.005468 -0.000316 -0.000316  6.186879e+06   
3    -0.008472  0.000470 -0.013179  0.006702  0.006702  6.259836e+06   
4    -0.005908  0.000000 -0.006933  0.002141  0.002141  6.302723e+06   
...        ...       ...       ...       ...       ...           ...   
2484 -0.003831  0.000000 -0.003831  0.002803  0.002803  2.985133e+07   
2485 -0.003040  0.004543 -0.003249  0.004531  0.004531  3.001950e+07   
2486  0.010826  0.011097 -0.000710 -0.008908 -0.008908  2.970274e+07   
2487  0.000998  0.003413 -0.003113 -0.001190 -0.001190  2.962466e+07   
2488  0.010598  0.010919 -0.002086 -0.009015 -0.009015  2.928837e+07   

                6             7             8             9       